In [1]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/My Drive/Colab Notebooks/cotton/Datasets/train'
valid_path = '/content/drive/My Drive/Colab Notebooks/cotton/Datasets/test'

In [3]:
#Import the InceptionV3 library as shown below and add preprocessing layer to the front of InceptionV3
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 2s 0us/step


In [5]:
for layer in inception.layers:
  layer.trainable = False

In [6]:
# useful for getting number of output classes
folders = glob('/content/drive/My Drive/Colab Notebooks/cotton/Datasets/train/*')
len(folders)
folders[0]

'/content/drive/My Drive/Colab Notebooks/cotton/Datasets/train/diseased cotton leaf'

In [7]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 111, 111, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [16]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [9]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/cotton/Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1951 images belonging to 4 classes.


In [11]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/cotton/Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 18 images belonging to 4 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20
37/61 [=================>............] - ETA: 9:06 - loss: 2.7491 - accuracy: 0.6681

In [12]:
# plot the loss
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='INCEPTION V3 train loss')
plt.plot(r.history['val_loss'], label='INCEPTION V3  val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy

plt.plot(r.history['accuracy'], label='INCEPTION V3  acc')
plt.plot(r.history['val_accuracy'], label='INCEPTION V3  val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

NameError: ignored

In [ ]:
# save it as a h5 file
from tensorflow.keras.models import load_model
model.save('/content/drive/My Drive/Colab Notebooks/cotton/model_inception.h5')

In [13]:
y_pred = model.predict(test_set)

In [14]:
import numpy as np
y_pred= np.argmax(y_pred, axis=1)
y_pred

array([0, 2, 3, 2, 1, 0, 2, 3, 2, 0, 2, 3, 1, 3, 2, 3, 3, 2])

In [ ]:
pred_img = load_img('/content/drive/My Drive/Colab Notebooks/cotton/Datasets/test/fresh cotton plant/dsd (632).jpg',target_size=(224,224))

In [ ]:
pred_img = img_to_array(pred_img)
pred_img

array([[[ 77.,  92.,  33.],
        [ 82.,  95.,  41.],
        [ 93., 101.,  50.],
        ...,
        [188., 240., 113.],
        [186., 239., 107.],
        [184., 237., 105.]],

       [[ 97., 107.,  47.],
        [ 88.,  93.,  37.],
        [ 94.,  98.,  47.],
        ...,
        [187., 244., 111.],
        [184., 241., 112.],
        [181., 240., 111.]],

       [[ 93.,  99.,  37.],
        [ 92.,  95.,  38.],
        [ 91.,  95.,  36.],
        ...,
        [178., 242., 106.],
        [185., 245., 111.],
        [174., 238., 102.]],

       ...,

       [[ 96., 120.,  58.],
        [123., 143.,  80.],
        [132., 148.,  83.],
        ...,
        [132., 223.,  40.],
        [129., 221.,  34.],
        [132., 224.,  37.]],

       [[ 92., 114.,  50.],
        [115., 135.,  72.],
        [118., 133.,  68.],
        ...,
        [119., 215.,  29.],
        [132., 223.,  40.],
        [126., 218.,  31.]],

       [[126., 138.,  74.],
        [145., 155.,  92.],
        [136., 1

In [ ]:
pred_img = pred_img/255
pred_img= np.expand_dims(pred_img,axis=0)
finalimage=preprocess_input(pred_img)
a=np.argmax(model.predict(finalimage), axis=1)

In [ ]:
if preds==0:
        print("The leaf is diseased cotton leaf")
elif preds==1:
        print("The leaf is diseased cotton plant")
elif preds==2:
        print("The leaf is fresh cotton leaf")
else:
        print("The leaf is fresh cotton plant")




The leaf is diseased cotton leaf
